<a href="https://colab.research.google.com/github/genilsonalmeida/escapeWithGemini/blob/main/EscapeWithGeminiAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
!pip install -q -U google-generativeai

In [38]:
import google.generativeai as genai
from google.colab import userdata

GOOGLE_API_KEY= userdata.get('google_api_key')
genai.configure(api_key=GOOGLE_API_KEY)

global generation_config
generation_config = {
  "temperature": 0.8
}

safety_settings={
    'HATE': 'BLOCK_NONE',
    'HARASSMENT': 'BLOCK_NONE',
    'SEXUAL' : 'BLOCK_NONE',
    'DANGEROUS' : 'BLOCK_NONE'
    }

model = genai.GenerativeModel(model_name="gemini-1.0-pro",
                          generation_config=generation_config,
                          safety_settings=safety_settings)

In [39]:
def exibir_menu():
    print("\n-------------------------")
    print("Jogo de Escape em Python")
    print("---------------------------")
    print("1. Iniciar Jogo")
    print("2. Regras do Jogo")
    print("3. Sair do Jogo")
    print("-----------------------")


In [48]:
def iniciar_jogo():
    print("\nInicio do jogo:")
    nivel = game_level()

    # Gera a sala inicial com base no nível
    sala_atual = gerar_sala(nivel)

    # Loop principal do jogo
    while True:
        # Exibe a descrição da sala e os objetos presentes
        print("\n" + sala_atual["descricao"])
        print("\n Projeto:     " + sala_atual["objetos"])
        for i, objeto in enumerate(sala_atual["objetos"]):
            print(f"{i+1}. {objeto}")

        # Obtém a ação do jogador
        acao_usuario = input("Digite o número do objeto que deseja interagir (ou 0 para sair): ")

        # Se o jogador deseja sair
        if acao_usuario == "0":
            print("\nObrigado por jogar!")
            break

        # Se o jogador escolher um objeto
        elif 0 < int(acao_usuario) <= len(sala_atual["objetos"]):
            objeto_escolhido = sala_atual["objetos"][int(acao_usuario) - 1]

            # Gera a resposta do Gemini com base na temperatura do nível e na interação com o objeto
            response = model.generate_content(f"Interagir com {objeto_escolhido}.")
            resposta_gemini = response.text
            print("\n" + resposta_gemini)

            # Verifica se a resposta atende ao critério de vitória secreto
            if sala_atual["criterio_vitoria"].lower() in resposta_gemini.lower():
                print("\nParabéns! Você escapou da sala!")
                break

            # Atualiza a sala com base na interação
            sala_atual = atualizar_sala(sala_atual, objeto_escolhido, resposta_gemini, nivel)

        # Se o jogador digitar uma opção inválida
        else:
            print("Opção inválida. Tente novamente.")

In [41]:
def gerar_sala(nivel):
    descricao = model.generate_content("Descreva uma sala de escape.").text
    objetos = model.generate_content("Liste os objetos presentes na sala.").text
    enigma = model.generate_content("Crie um enigma para a sala.").text
    criterio_vitoria = model.generate_content("Defina o critério de vitória para a sala (secreto).").text

    return {
        "descricao": descricao,
        "objetos": objetos,
        "enigma": enigma,
        "criterio_vitoria": criterio_vitoria
    }

In [42]:
def atualizar_sala(sala_atual, objeto_escolhido, resposta_gemini, nivel):
    """
    Atualiza a sala do jogo de escape com base na interação do jogador e no nível de dificuldade.

    Argumentos:
      sala_atual (dict): Dicionário contendo as informações da sala atual.
      objeto_escolhido (str): Nome do objeto com o qual o jogador interagiu.
      resposta_gemini (str): Resposta gerada pelo Gemini para a interação.
      nivel (int): Nível de dificuldade do jogo (1 - Fácil, 2 - Intermediário, 3 - Difícil).

    Retorno:
      dict: Dicionário atualizado da sala do jogo.
    """

    # Remove o objeto interagido da lista de objetos da sala (exceto em níveis fáceis)
    if nivel != 1:
        sala_atual["objetos"] = [objeto for objeto in sala_atual["objetos"] if objeto != objeto_escolhido]

    # Adiciona pistas ou informações adicionais sobre o enigma da sala (dependendo do nível)
    if nivel == 2:  # Nível Intermediário
        pista = model.generate_content("Fornecer pista sutil sobre o enigma.").text
        sala_atual["descricao"] += f"\n\n**Pista:** {pista}"
    elif nivel == 3:  # Nível Difícil
        novo_enigma = model.generate_content("Criar um novo enigma mais desafiador.").text
        sala_atual["enigma"] = novo_enigma

    # Retorna a sala atualizada
    return sala_atual



In [43]:
def game_level():
    print("\n-------------------------")
    print("Qual a dificuldade do jogo?")
    print("---------------------------")
    print("1. Fácil")
    print("2. Intermediário")
    print("3. Difícil")
    print("-----------------------")

    while True:
      nivel_escolhido = input("Digite o número da dificuldade desejada: ")

      if nivel_escolhido == "1":
        generation_config.update({"temperature": 0.8})  # Fácil - alta temperatura, alto candidate_count
        return 1
      elif nivel_escolhido == "2":
        generation_config.update({"temperature": 0.6})  # Intermediário - temperatura média, candidate_count médio
        return 2
      elif nivel_escolhido == "3":
        generation_config.update({"temperature": 0.4})  # Difícil - baixa temperatura, baixo candidate_count
        return 3
      else:
        print("Opção inválida. Tente novamente.")

In [44]:
def mostrar_regras():
    print("\nRegras do Jogo:")
    print("1. Você está preso em uma sala e precisa escapar.")
    print("2. Para escapar, você precisa resolver enigmas e encontrar a chave escondida.")
    print("3. Use os comandos 'examinar' e 'interagir' para explorar a sala e interagir com objetos.")
    print("4. Boa sorte!")

In [45]:
def sair_jogo():
    while True:
        opcao = input("Deseja mesmo sair do jogo? \nSim(S) ou Não(N): ").lower()
        if opcao == "sim" or opcao == "s":
            print("Obrigado por jogar!")
            return False
        elif opcao == "não" or opcao == "n" or opcao == "nao":
            return True
        else:
            print("Opção inválida. Tente novamente.")

In [47]:
while True:
  exibir_menu()

  opcao = input("Digite sua opção: ")

  if opcao == "1":
    iniciar_jogo()
  elif opcao == "2":
    mostrar_regras()
  elif opcao == "3":
    if not sair_jogo():
      break
  else:
    print("Opção inválida. Tente novamente.")


-------------------------
Jogo de Escape em Python
---------------------------
1. Iniciar Jogo
2. Regras do Jogo
3. Sair do Jogo
-----------------------
Digite sua opção: 1

Inicio do jogo:

-------------------------
Qual a dificuldade do jogo?
---------------------------
1. Fácil
2. Intermediário
3. Difícil
-----------------------
Digite o número da dificuldade desejada: 1

Uma sala de escape é um espaço imersivo projetado para testar as habilidades de resolução de quebra-cabeças e trabalho em equipe de um grupo. Ela normalmente consiste em:

* **Uma história envolvente:** Os participantes assumem papéis em uma narrativa cativante que os conduz pela sala.
* **Uma série de desafios:** Os participantes enfrentam uma variedade de quebra-cabeças, enigmas e desafios físicos que testam sua inteligência, habilidades de observação e capacidade de trabalhar juntos.
* **Um cenário temático:** A sala é decorada e mobiliada para criar um ambiente específico, como um laboratório científico, uma m

KeyboardInterrupt: Interrupted by user